<a href="https://colab.research.google.com/github/Harshanand7/ASVA.AI/blob/main/distilbert_model_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers -U

In [ ]:
!pip install datasets

In [ ]:
import pandas as pd

In [ ]:
from datasets import load_dataset

dataset = load_dataset("christophsonntag/OLID")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
df = dataset["train"].to_pandas()
df=df[['cleaned_tweet','subtask_a']]
df.head()

,cleaned_tweet,subtask_a
0,ask native americans take,OFF
1,go home drunk maga trump 2020,OFF
2,amazon investigating chinese employees selling...,NOT
3,someone vetaken piece shit volcano,OFF
4,obama wanted liberals illegals move red states,NOT


In [ ]:


# Map 'OFF' to 1 and 'NOT' to 0 in the 'subtask_a' column
df['subtask_a'] = df['subtask_a'].map({'OFF': 1, 'NOT': 0})

# Display the first few rows to confirm the changes
print(df.head())

                                       cleaned_tweet  subtask_a
0                          ask native americans take          1
1                      go home drunk maga trump 2020          1
2  amazon investigating chinese employees selling...          0
3                 someone vetaken piece shit volcano          1
4     obama wanted liberals illegals move red states          0


In [ ]:
output_file = 'modified_tweets.csv'
df.to_csv(output_file, index=False)

In [ ]:

data = pd.read_csv("/content/modified_tweets.csv",  engine="python")
data.head()
data=data[['cleaned_tweet','subtask_a']]

In [ ]:

data = data[0:1000]
data.head()

,cleaned_tweet,subtask_a
0,ask native americans take,1
1,go home drunk maga trump 2020,1
2,amazon investigating chinese employees selling...,0
3,someone vetaken piece shit volcano,1
4,obama wanted liberals illegals move red states,0


In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
import torch
from transformers import TrainingArguments, Trainer
from transformers import BertTokenizer, BertForSequenceClassification

In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import DistilBertTokenizer
from transformers import DistilBertForSequenceClassification
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

# train_encodings = tokenizer(train_texts, truncation = True, padding = True  )
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=5)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
model

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
 

In [ ]:
model = model.to('cuda')

In [ ]:
sample_data = ["I am eating","I am playing "]
tokenizer(sample_data, padding=True, truncation=True, max_length=512)

{'input_ids': [[101, 1045, 2572, 5983, 102], [101, 1045, 2572, 2652, 102]], 'attention_mask': [[1, 1, 1, 1, 1], [1, 1, 1, 1, 1]]}

In [ ]:
X = list(data["cleaned_tweet"])
y = list(data["subtask_a"])
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2,stratify=y)
X_train_tokenized = tokenizer(
    [str(text) for text in X_train],  # Convert each element in X_train to a string
    padding=True,
    truncation=True,
    max_length=512
)
X_val_tokenized = tokenizer(
    [str(text) for text in X_val],  # Convert each element in X_val to a string
    padding=True,
    truncation=True,
    max_length=512
)

In [ ]:
X_train_tokenized.keys()

dict_keys(['input_ids', 'attention_mask'])

In [ ]:
print(X_train_tokenized['attention_mask'][0])

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [ ]:
len(X_train),len(X_val)

(400, 100)

In [ ]:
# Create torch dataset
class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels=None):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        if self.labels:
            item["labels"] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.encodings["input_ids"])

In [ ]:
train_dataset = Dataset(X_train_tokenized, y_train)
val_dataset = Dataset(X_val_tokenized, y_val)

In [ ]:
train_dataset[5]

{'input_ids': tensor([ 101, 2829, 2066,  102,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0]),
 'attention_mask': tensor([1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0]),
 'labels': tensor(0)}

In [ ]:
def compute_metrics(p):
    print(type(p))
    pred, labels = p
    pred = np.argmax(pred, axis=1)

    accuracy = accuracy_score(y_true=labels, y_pred=pred)
    recall = recall_score(y_true=labels, y_pred=pred)
    precision = precision_score(y_true=labels, y_pred=pred)
    f1 = f1_score(y_true=labels, y_pred=pred)

    return {"accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1}

In [ ]:
!pip install accelerate -U

In [ ]:
# Define Trainer
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
args = TrainingArguments(
    output_dir="output",
    num_train_epochs=2,
    per_device_train_batch_size=8,
    logging_steps=50,  # Log every 10 steps
    report_to="none"  # Disable other reporting to focus on logs


)
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)

In [ ]:
trainer.train()

Step,Training Loss
50,0.037100
100,0.000300


TrainOutput(global_step=100, training_loss=0.018679671669378877, metrics={'train_runtime': 11.2453, 'train_samples_per_second': 71.141, 'train_steps_per_second': 8.893, 'total_flos': 10142577816000.0, 'train_loss': 0.018679671669378877, 'epoch': 2.0})

In [ ]:
trainer.evaluate()
#0.795

<class 'transformers.trainer_utils.EvalPrediction'>


{'eval_loss': 2.745753765106201,
 'eval_accuracy': 0.74,
 'eval_precision': 0.6129032258064516,
 'eval_recall': 0.5757575757575758,
 'eval_f1': 0.59375,
 'eval_runtime': 0.2613,
 'eval_samples_per_second': 382.664,
 'eval_steps_per_second': 49.746,
 'epoch': 2.0}

In [ ]:
np.set_printoptions(suppress=True)

In [ ]:
text = "That was good point"

inputs = tokenizer(text,padding = True, truncation = True, return_tensors='pt').to('cuda')
outputs = model(**inputs)
print(outputs)
predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
print(predictions)
predictions = predictions.cpu().detach().numpy()
predictions
non_offensive_prob_1 = predictions[0][0]
#The output is an array of two elements where the first element is the probability of the test to be not offensive
#and the second one is the probability of text as  offensive

SequenceClassifierOutput(loss=None, logits=tensor([[ 9.7028, -3.0790, -4.6205, -4.6389, -5.1260]], device='cuda:0',
       grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)
tensor([[1.0000e+00, 2.8116e-06, 6.0187e-07, 5.9087e-07, 3.6304e-07]],
       device='cuda:0', grad_fn=<SoftmaxBackward0>)


In [ ]:
if (non_offensive_prob_1>=0.67):
  print("The given text is likely to be classified as NON Offensive text")
else:
  print("The given text is probably OFFENSIVE text")

The given text is likely to be classified as NON Offensive text


In [ ]:
trainer.save_model('CustomModel')

In [ ]:
model_2 = DistilBertForSequenceClassification.from_pretrained("CustomModel")
model_2.to('cuda')

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
 

In [ ]:

text = "you are a fool"
inputs = tokenizer(text,padding = True, truncation = True, return_tensors='pt').to('cuda')
outputs = model_2(**inputs)
predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
predictions = predictions.cpu().detach().numpy()
predictions
non_offensive_prob = predictions[0][0]
predictions
#The output is an array where the first element is the probability of the test to be not offensive
#and the second one is the probability of text as  offensive



array([[0.00000862, 0.9999865 , 0.0000017 , 0.0000016 , 0.00000156]],
      dtype=float32)

In [ ]:
if (non_offensive_prob>=0.67):
  print("The given text is likely to be classified as NON Offensive text")
else:
  print("The given text is probably OFFENSIVE text")

The given text is probably OFFENSIVE text
